In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from langchain_google_genai import ChatGoogleGenerativeAI
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage
from langgraph_sdk import get_client

import logging
import uuid


In [3]:
client = get_client(url="http://localhost:2024")

In [4]:

user_query = ["what are some recent news of Nvidia?"]

user_input_messages = user_query



MAX_CACHE_SIZE = 2

streaming_llm_agents = [*TEAM_MEMBERS, "planner", "coordinator"]

# Reset coordinator cache at the start of each workflow
global coordinator_cache
coordinator_cache = []
global is_handoff_case
is_handoff_case = False

thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)



HumanMessage(
    content='what are some recent news of Nvidia?',
    additional_kwargs={},
    response_metadata={},
    id='2f397efb-6ece-4a5d-b36e-d003f2203fc0',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants to know about the most recent news related to Nvidia. I need to gather up-to-dat'
        'e news articles and summarize the key developments.",\n'
        '  "title": "Recent News of Nvidia",\n'
        '  "steps": [\n'
        '    {\n'
        '      "agent_name": "researcher",\n'
        '      "title": "Collect and Summarize Recent News about Nvidia",\n'
        '      "description": "The researcher will search for the most recent news articles and updates about Nvidia, '
        'summarizing the key points and developments in a Markdown report."\n'
        '    },\n'
        '    {\n'
        '      "agent_name": "reporter",\n'
        '      "title": "Compile and Present a Professional Report on Nvidia\'s Recent News",\n'
        '     

In [5]:
current_state = await client.threads.get_state(thread["thread_id"])

In [6]:
current_state

{'values': {'messages': [{'content': 'what are some recent news of Nvidia?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '2f397efb-6ece-4a5d-b36e-d003f2203fc0',
    'example': False},
   {'content': '{\n  "thought": "The user wants to know about the most recent news related to Nvidia. I need to gather up-to-date news articles and summarize the key developments.",\n  "title": "Recent News of Nvidia",\n  "steps": [\n    {\n      "agent_name": "researcher",\n      "title": "Collect and Summarize Recent News about Nvidia",\n      "description": "The researcher will search for the most recent news articles and updates about Nvidia, summarizing the key points and developments in a Markdown report."\n    },\n    {\n      "agent_name": "reporter",\n      "title": "Compile and Present a Professional Report on Nvidia\'s Recent News",\n      "description": "The reporter will take the researcher\'s findings and write a clear, professio

In [7]:
from IPython.display import Markdown, display


# Display the formatted markdown
display(Markdown(current_state['values']['messages'][-1]['content']))

Response from reporter:

<response>
# Recent News of Nvidia

## Executive Summary

This report provides an overview of the most recent news concerning Nvidia as of April 17, 2025. The company is currently experiencing significant challenges due to new U.S. government restrictions on AI chip exports to China, resulting in a notable stock decline and increased regulatory scrutiny. Despite ongoing product innovation, these positive developments are being overshadowed by geopolitical and compliance risks.

---

## Key Findings

- **Nvidia’s stock dropped 6.9%** following the announcement of a potential $5.5 billion financial impact from new U.S. export restrictions on AI chips to China.
- **CEO Jensen Huang made a surprise visit to Beijing** shortly after the restrictions, signaling Nvidia’s continued interest in the Chinese market.
- **U.S.-China trade tensions are intensifying**, with Nvidia identified as a major casualty due to its reliance on the Chinese market for AI chip sales.
- **U.S. lawmakers have launched an investigation** into Nvidia’s chip sales in China, increasing legal and compliance risks.
- **Nvidia launched new GeForce RTX 5060 and 5060 Ti graphics cards**, but these product releases have been overshadowed by the export ban and related news.

---

## Detailed Analysis

### 1. Stock Drop Due to AI Export Ban

- **Event**: Nvidia’s stock fell by 6.9% after the company disclosed a potential $5.5 billion loss resulting from new U.S. government restrictions on AI chip exports to China.
- **Details**: The ban specifically targets Nvidia’s H20 GPUs, which were designed for the Chinese market. The indefinite nature of the restrictions has created uncertainty regarding future revenue from China.
- **Impact**: Short-term negative sentiment among investors and increased volatility in Nvidia’s stock price.
- **Source**: [Watcher Guru](https://watcher.guru/news/wednesday-wipeout-nvidia-nvda-drops-6-9-after-5-5b-hit-from-ai-export-ban)

### 2. CEO Jensen Huang’s Visit to Beijing

- **Event**: Nvidia’s CEO, Jensen Huang, made an unannounced visit to Beijing after the U.S. imposed new chip sales restrictions.
- **Details**: Huang met with the founder of Chinese AI company DeepSeek and attended events organized by a trade group. The visit attracted significant attention on Chinese social media.
- **Impact**: Demonstrates Nvidia’s commitment to the Chinese market despite regulatory challenges.
- **Source**: [The Guardian](https://www.theguardian.com/technology/2025/apr/17/nvidias-ceo-surprise-visit-beijing-restricts-chip-sales-china)

### 3. U.S.-China Trade Tensions

- **Event**: Nvidia is highlighted as a major casualty in the ongoing U.S.-China trade war.
- **Details**: The new export restrictions threaten Nvidia’s chip sales in China, and the company faces increased scrutiny from U.S. lawmakers.
- **Impact**: Heightened regulatory and geopolitical risks for Nvidia’s business in Asia.
- **Source**: [Morning Brew Daily Podcast](https://www.podtrac.com/pts/redirect.mp3/pdst.fm/e/tracking.swap.fm/track/bHsDpvy53mYwJpnc1UL5/traffic.megaphone.fm/MOBI9749366598.mp3?updated=1744889191)

### 4. U.S. Lawmakers Investigate Nvidia’s China Sales

- **Event**: The House Select Committee on the Chinese Communist Party has opened an investigation into Nvidia’s chip sales in China.
- **Details**: The investigation aims to determine whether Nvidia’s sales practices violate U.S. export rules.
- **Impact**: Increased legal and compliance risks for Nvidia.
- **Source**: [New York Times](https://www.nytimes.com/2025/04/17/business/nvidia-china-tariffs-chips-jensen-huang.html)

### 5. Product Launches Overshadowed

- **Event**: Nvidia launched new GeForce RTX 5060 and 5060 Ti graphics cards.
- **Details**: The new products offer significant performance improvements and price reductions.
- **Impact**: Positive for product innovation, but the news has been overshadowed by the export ban and related regulatory issues.
- **Source**: [Watcher Guru](https://watcher.guru/news/wednesday-wipeout-nvidia-nvda-drops-6-9-after-5-5b-hit-from-ai-export-ban)

---

## Summary Table

| Date       | Headline/Topic                                      | Sentiment   |
|------------|-----------------------------------------------------|-------------|
| Apr 17, 25 | $5.5B hit from AI export ban, stock drops 6.9%      | Negative    |
| Apr 17, 25 | CEO Jensen Huang visits Beijing                     | Neutral/+   |
| Apr 17, 25 | U.S.-China trade war impacts Nvidia                 | Negative    |
| Apr 17, 25 | U.S. lawmakers investigate China sales              | Negative    |
| Apr 17, 25 | New RTX 5060/5060 Ti launches                       | Positive    |

---

## Conclusions and Recommendations

### Conclusions

- Nvidia is currently facing significant regulatory and geopolitical challenges, primarily due to new U.S. export restrictions on AI chips to China.
- The company’s financial outlook and stock performance have been negatively affected by these developments.
- Ongoing product innovation continues, but is not sufficient to offset the negative impact of regulatory and trade issues.

### Recommendations

- **For Investors**: Monitor ongoing regulatory developments and Nvidia’s response to export restrictions, as these will significantly impact the company’s financial performance.
- **For Nvidia**: Continue to diversify markets and product lines to reduce reliance on China and mitigate regulatory risks.
- **For Stakeholders**: Stay informed about the outcomes of U.S. government investigations and potential changes in trade policy.

---

**Sources**:
- [Watcher Guru](https://watcher.guru/news/wednesday-wipeout-nvidia-nvda-drops-6-9-after-5-5b-hit-from-ai-export-ban)
- [The Guardian](https://www.theguardian.com/technology/2025/apr/17/nvidias-ceo-surprise-visit-beijing-restricts-chip-sales-china)
- [Morning Brew Daily Podcast](https://www.podtrac.com/pts/redirect.mp3/pdst.fm/e/tracking.swap.fm/track/bHsDpvy53mYwJpnc1UL5/traffic.megaphone.fm/MOBI9749366598.mp3?updated=1744889191)
- [New York Times](https://www.nytimes.com/2025/04/17/business/nvidia-china-tariffs-chips-jensen-huang.html)
- [Seeking Alpha](https://seekingalpha.com/article/4775843-clearbridge-select-strategy-q1-2025-commentary)
</response>

*Please execute the next step.*